In [23]:
import gymnasium
import highway_env
# import finite-mdp

%load_ext tensorboard
import sys
from tqdm.notebook import trange
!pip install tensorboardx gym pyvirtualdisplay
# doesn't work cause not linux
# !apt-get install -y xvfb ffmpeg
!git clone https://github.com/Farama-Foundation/HighwayEnv.git 2> /dev/null
!git clone https://github.com/eleurent/finite-mdp.git 2> /dev/null
sys.path.insert(0, '/content/HighwayEnv/scripts/')
# from utils import record_videos, show_videos



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:

    # def is_finite_mdp(env):

finite_mdp_prep = __import__("finite-mdp.finite_mdp")
# print(finite_mdp_prep.__file__)
# g = finite_mdp_prep.finite_mdp
# f = g.envs.finite_mdp_envs
# finite_mdp = __import__("finite-mdp.finite_mdp.envs.finite_mdp_env")


None


In [ ]:
import gymnasium as gym
import highway_env
from stable_baselines3 import DQN

class ValueIteration:
    def __init__(self):
        pass

# Visualization utils
